In [103]:
import pandas as pd
import matplotlib.pyplot as plt

In [104]:
tracks = ["porto", "simple", "u", "tornado", "race_track", "zurich"]
csvs = {}

for track in tracks:
    file_name = track + ".csv"
    csv = pd.read_csv(file_name)
    csvs[track] = csv

In [105]:
csvs["race_track"]

,algorithm,circuit name,number of actions,from waypoint,lookahead,finished within time limit,found solution,open nodes,expanded nodes,time to finish,travelled distance,time step,discretization,success rate,number of repetitions,average computation time in ms,computation time variance
0,hybrid_astar,race_track,44,0,9,yes,no,120899,7786,0.00,0.0000,0.04,x-1.44222_y-1.44222-rpm-875-theta-20,1,10,351.7,37.01
1,hybrid_astar,race_track,44,0,9,yes,no,87227,5676,0.00,0.0000,0.04,x-1.80278_y-1.80278-rpm-875-theta-20,1,10,263.2,14.76
2,sehs,race_track,44,0,9,yes,yes,169002,9718,25.52,82.6613,0.04,c-164-rpm-875-theta-20,1,10,598.1,43.29
3,hybrid_astar,race_track,84,0,9,yes,no,221867,7781,0.00,0.0000,0.04,x-1.44222_y-1.44222-rpm-875-theta-20,1,10,658.8,67.56
4,hybrid_astar,race_track,84,0,9,yes,no,133413,4637,0.00,0.0000,0.04,x-1.80278_y-1.80278-rpm-875-theta-20,1,10,415.0,13.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,hybrid_astar,race_track,420,0,9,no,no,4486688,28636,0.00,0.0000,0.04,x-1.80278_y-1.80278-rpm-218.75-theta-6.92308,0,10,10085.0,0.00
239,sehs,race_track,420,0,9,no,no,3726704,21187,0.00,0.0000,0.04,c-164-rpm-218.75-theta-6.92308,0,10,10089.0,0.00
240,hybrid_astar,race_track,620,0,9,no,no,4264634,17970,0.00,0.0000,0.04,x-1.44222_y-1.44222-rpm-218.75-theta-6.92308,0,10,10105.0,0.00
241,hybrid_astar,race_track,620,0,9,no,no,4196989,17736,0.00,0.0000,0.04,x-1.80278_y-1.80278-rpm-218.75-theta-6.92308,0,10,10094.0,0.00


In [106]:
print(csvs["race_track"].columns)

Index(['algorithm', 'circuit name', 'number of actions', 'from waypoint',
       'lookahead', 'finished within time limit', 'found solution',
       'open nodes', 'expanded nodes', 'time to finish', 'travelled distance',
       'time step', 'discretization', 'success rate', 'number of repetitions',
       'average computation time in ms', 'computation time variance'],
      dtype='object')


In [107]:
class PDF(object):
  def __init__(self, pdf, size=(400,300)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

# example: hybrid_astar_porto_a-124_f-0_l-5_t-0.04_d-x-1.44222_y-1.44222-rpm-218.75-theta-10_trajectory.pdf
def base_image_path(data):
    r = data.iloc[0]
    return "{}_{}_a-{}_f-0_l-{}_t-{}_d-{}".format(r["algorithm"], r["circuit name"], r["number of actions"], r["lookahead"], r["time step"], r["discretization"])
    
def trajectory_image(data):
    name = base_image_path(data) + "_trajectory.pdf"
    return name

def stats_image(data):
    name = base_image_path(data) + "_actuators.pdf"
    return name

In [143]:
from IPython.display import display

def analyze_alg(data):
    least_opened_nodes = min(data["open nodes"])
    attempt_with_least_opened_nodes = data[data["open nodes"] == least_opened_nodes]
    time_per_open_node = data["average computation time in ms"].iloc[0] / least_opened_nodes
    
    print("\nleast expanded nodes: {} ({}ms per open  node)".format(least_opened_nodes, time_per_open_node))
    display(attempt_with_least_opened_nodes)
    print(trajectory_image(attempt_with_least_opened_nodes))
#     display(PDF(trajectory_image(attempt_with_least_opened_nodes)))
    print(stats_image(attempt_with_least_opened_nodes))
#     display(PDF(stats_image(attempt_with_least_opened_nodes)))
    
    min_computation_time = min(data["average computation time in ms"])
    fastest = data[data["average computation time in ms"] == min_computation_time]
    
    print("\nfastest computation: {}ms".format(min_computation_time))
    display(fastest)
    print(trajectory_image(fastest))
#     display(PDF(trajectory_image(fastest)))
    print(stats_image(fastest))
#     display(PDF(stats_image(fastest)))
    
    best_lap_time = min(data["time to finish"])
    best_lap = data[data["time to finish"] == best_lap_time]
    
    print("\nbest lap: {}s".format(best_lap_time))
    display(best_lap)
    print(trajectory_image(best_lap))
#     display(PDF(trajectory_image(best_lap)))
    print(stats_image(best_lap))
#     display(PDF(stats_image(best_lap)))

def analyze_track(track):
    print("RESULTS FOR {}".format(track))
    print("=======================")
    
    data = csvs[track]
    print("total tests: ", len(data))
    
    finished = data[data["finished within time limit"] == "yes"]
    print("finished within time limit: ", len(finished))

    successful = data[data["found solution"] == "yes"]
    print("found solution: ", len(successful))

    sehs = successful[data["algorithm"] == "sehs"]
    hybrid_astar = successful[successful["algorithm"] == "hybrid_astar"]
    print("found solution SEHS: ", len(sehs))
    print("found solution Hybrid A*: ", len(hybrid_astar))
    print()
    
    if len(sehs) > 0:
        print("SEHS:")
        analyze_alg(sehs)
        print()

    if len(hybrid_astar) > 0:
        print("Hybrid A*:")
        analyze_alg(hybrid_astar)

    print()
    print()

In [144]:
for track in tracks:
    analyze_track(track)

RESULTS FOR porto
total tests:  243
finished within time limit:  219
found solution:  216
found solution SEHS:  70
found solution Hybrid A*:  146

SEHS:

least expanded nodes: 80763 (0.0031400517563735866ms per open  node)


/usr/lib/python3/dist-packages/ipykernel_launcher.py:48: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


,algorithm,circuit name,number of actions,from waypoint,lookahead,finished within time limit,found solution,open nodes,expanded nodes,time to finish,travelled distance,time step,discretization,success rate,number of repetitions,average computation time in ms,computation time variance
2,sehs,porto,44,0,5,yes,yes,80763,4864,13.16,39.6191,0.04,c-71-rpm-875-theta-20,1,10,253.6,10.84


sehs_porto_a-44_f-0_l-5_t-0.04_d-c-71-rpm-875-theta-20_trajectory.pdf
sehs_porto_a-44_f-0_l-5_t-0.04_d-c-71-rpm-875-theta-20_actuators.pdf

fastest computation: 253.6ms


,algorithm,circuit name,number of actions,from waypoint,lookahead,finished within time limit,found solution,open nodes,expanded nodes,time to finish,travelled distance,time step,discretization,success rate,number of repetitions,average computation time in ms,computation time variance
2,sehs,porto,44,0,5,yes,yes,80763,4864,13.16,39.6191,0.04,c-71-rpm-875-theta-20,1,10,253.6,10.84


...

hybrid_astar_zurich_a-44_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-437.5-theta-10_trajectory.pdf
hybrid_astar_zurich_a-44_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-437.5-theta-10_actuators.pdf

fastest computation: 1041.1ms


,algorithm,circuit name,number of actions,from waypoint,lookahead,finished within time limit,found solution,open nodes,expanded nodes,time to finish,travelled distance,time step,discretization,success rate,number of repetitions,average computation time in ms,computation time variance
109,hybrid_astar,zurich,44,0,12,yes,yes,488873,29275,41.28,123.667,0.04,x-1.80278_y-1.80278-rpm-437.5-theta-10,1,10,1041.1,60.29


hybrid_astar_zurich_a-44_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-437.5-theta-10_trajectory.pdf
hybrid_astar_zurich_a-44_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-437.5-theta-10_actuators.pdf

best lap: 38.52s


,algorithm,circuit name,number of actions,from waypoint,lookahead,finished within time limit,found solution,open nodes,expanded nodes,time to finish,travelled distance,time step,discretization,success rate,number of repetitions,average computation time in ms,computation time variance
145,hybrid_astar,zurich,88,0,12,yes,yes,1922591,60606,38.52,120.091,0.04,x-1.80278_y-1.80278-rpm-218.75-theta-10,1,10,3811.0,926.0


hybrid_astar_zurich_a-88_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-218.75-theta-10_trajectory.pdf
hybrid_astar_zurich_a-88_f-0_l-12_t-0.04_d-x-1.80278_y-1.80278-rpm-218.75-theta-10_actuators.pdf




In [203]:
import re

def print_performance(r):
    return "\\num{{{0}}} & \\SI{{{1:.2f}}}{{\milli\second}} & \\SI{{{2:.2f}}}{{\meter}} & \\SI{{{3:.2f}}}{{\second}}".format(
            r["open nodes"],
            r["average computation time in ms"],
            r["travelled distance"],
            r["time to finish"])

def print_settings_sehs(r):    
    # c-126-rpm-218.75-theta-6.92308
    circles = int(re.search(r"^c-([^-]+)-", r["discretization"]).group(1))
    rpm_levels = 17500 / float(re.search(r"-rpm-([^-]+)-", r["discretization"]).group(1))
    theta_levels = 360 / float(re.search(r"-theta-([^$]+)$", r["discretization"]).group(1))

    return "\\num{{{0}}} & \\num{{{1:.0f}}} & \\num{{{2:.0f}}} & \\num{{{3:.0f}}}".format(
            r["number of actions"],
            circles,
            rpm_levels,
            theta_levels)

def print_settings_hybrid_astar(r):    
    # x-1.4-y-1.4-rpm-218.75-theta-6.92308
    size = float(re.search(r"^x-([^-]+)_", r["discretization"]).group(1))
    rpm_levels = 17500 / float(re.search(r"-rpm-([^-]+)-", r["discretization"]).group(1))
    theta_levels = 360 / float(re.search(r"-theta-([^$]+)$", r["discretization"]).group(1))

    return "\\num{{{0}}} & \\num{{{1:.2f}}} & \\num{{{2:.0f}}} & \\num{{{3:.0f}}}".format(
            r["number of actions"],
            size,
            rpm_levels,
            theta_levels)

def print_sehs_row_for_latex(data):
    r = data.iloc[0]
    print("Actions & Circles & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \\")
    print("{} & {} \\\\".format(
            print_settings_sehs(r),
            print_performance(r)))

def print_hybrid_astar_row_for_latex(data):
    r = data.iloc[0]
    print("Actions & Grid cell size & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \\")
    print("{} & {} \\\\".format(
            print_settings_hybrid_astar(r),
            print_performance(r)))
    
def analyze_alg_for_latex(alg, data):
    least_opened_nodes = min(data["open nodes"])
    attempt_with_least_opened_nodes = data[data["open nodes"] == least_opened_nodes]
    
    if alg == "sehs":
        print_sehs_row_for_latex(attempt_with_least_opened_nodes)
    else:
        print_hybrid_astar_row_for_latex(attempt_with_least_opened_nodes)
    
    best_lap_time = min(data["time to finish"])
    best_lap = data[data["time to finish"] == best_lap_time]
    
    if alg == "sehs":
        print_sehs_row_for_latex(best_lap)
    else:
        print_hybrid_astar_row_for_latex(best_lap)

def analyze_track_for_latex(track):
    print("RESULTS FOR {}".format(track))
    print("=======================")
    
    data = csvs[track]    
    finished = data[data["finished within time limit"] == "yes"]
    successful = data[data["found solution"] == "yes"]
    sehs = successful[data["algorithm"] == "sehs"]
    hybrid_astar = successful[successful["algorithm"] == "hybrid_astar"]
    
    if len(sehs) > 0:
        print("SEHS:")
        analyze_alg_for_latex("sehs", sehs)
        print()

    if len(hybrid_astar) > 0:
        print("Hybrid A*")
        analyze_alg_for_latex("hybrid_astar", hybrid_astar)

    print()
for track in tracks:
    analyze_track_for_latex(track)

RESULTS FOR porto
SEHS:
Actions & Circles & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \
\num{44} & \num{71} & \num{20} & \num{18} & \num{80763} & \SI{253.60}{\milli\second} & \SI{39.62}{\meter} & \SI{13.16}{\second} \\
Actions & Circles & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \
\num{84} & \num{71} & \num{80} & \num{36} & \num{1054427} & \SI{2171.60}{\milli\second} & \SI{37.50}{\meter} & \SI{12.48}{\second} \\

Hybrid A*
Actions & Grid cell size & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \
\num{44} & \num{1.80} & \num{20} & \num{18} & \num{67301} & \SI{192.30}{\milli\second} & \SI{39.94}{\meter} & \SI{13.64}{\second} \\
Actions & Grid cell size & RPM levels & Heading angle sectors & Opened & Expanded & Search time & Distance & Lap time \
\num{248} & \num{1.44} & \num{20} & \num{36} & \num{838926} & \SI{2544.00}{\milli\second} & \SI{36.99

/usr/lib/python3/dist-packages/ipykernel_launcher.py:72: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
